In [7]:
import ccxt
import time
from itertools import permutations

# Replace with your actual Bybit API keys
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'

# Connect to Bybit API
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

# Arbitrage settings
SPREAD_THRESHOLD = 0.5 / 100  # 0.5% profit target
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
TRADING_FEE = 0.001  # 0.1% trading fee per trade

def get_tradable_tokens():
    """Fetches all tradable tokens on Bybit and filters out low-volume pairs"""
    try:
        markets = bybit.load_markets()
        tradable_tokens = []
        for symbol in markets:
            if '/USDT' in symbol and ':USDT' not in symbol:
                market_data = bybit.fetch_ticker(symbol)
                volume = market_data.get('quoteVolume', 0)  # Get 24h trading volume
                if volume and volume >= MIN_VOLUME:
                    tradable_tokens.append(symbol)

        print(f"✅ Found {len(tradable_tokens)} tradable tokens")  # Debug print
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens: {e}")
        return []

def get_price(symbol):
    """Fetch latest price for a given token if the market exists"""
    try:
        # Check if the market exists before fetching price
        if symbol in bybit.load_markets():
            ticker = bybit.fetch_ticker(symbol)
            
            # Extract the price correctly from Bybit's response format
            if "lastPrice" in ticker:
                return float(ticker["lastPrice"])
            elif "last" in ticker:
                return float(ticker["last"])
            elif "list" in ticker and len(ticker["list"]) > 0:
                return float(ticker["list"][0]["lastPrice"])

        print(f"⚠️ Market not found: {symbol} (Skipping)")
        return None
    except Exception as e:
        print(f"❌ Error fetching price for {symbol}: {e}")
        return None

def check_intra_exchange_arbitrage(symbol):
    """Checks for spot vs futures arbitrage opportunities within Bybit"""
    spot_price = get_price(symbol)
    futures_symbol = symbol.replace('/USDT', '/USDT:USDT')

    # Check if the futures market exists before fetching
    if futures_symbol not in bybit.load_markets():
        print(f"⚠️ Futures market not found for {symbol} (Skipping)")
        return

    futures_price = get_price(futures_symbol)

    if not spot_price or not futures_price:
        return

    spread = (futures_price - spot_price) / spot_price  # Calculate spread %
    net_profit = spread - (TRADING_FEE * 2)  # Adjust for fees

    if net_profit >= SPREAD_THRESHOLD:
        print(f"🔍 Intra-exchange Arbitrage Detected on Bybit for {symbol}")
        print(f"    ✅ Spot Price: {spot_price:.2f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.2f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        print("-" * 50)

def log_arbitrage(trade_path, expected_return, net_profit):
    """Logs detected arbitrage trades to a file"""
    with open("arbitrage_trades.log", "a") as log_file:
        log_file.write(f"{trade_path} | Expected: {expected_return:.4%} | Net Profit: {net_profit:.4%}\n")

def check_triangular_arbitrage(tradable_tokens):
    """Checks for triangular arbitrage opportunities within Bybit"""
    for route in permutations(tradable_tokens, 3):  # Ensure three different assets
        base_asset = route[0].split('/')[0]  # First asset in the path
        mid_asset = route[1].split('/')[0]
        final_asset = route[2].split('/')[1]  # Final conversion back to USDT

        if base_asset == final_asset:  # Ensure a valid cycle
            try:
                # Get price data for each leg
                first_leg_price = get_price(route[0])
                second_leg_price = get_price(route[1])
                third_leg_price = get_price(route[2])

                if first_leg_price and second_leg_price and third_leg_price:
                    # Convert using proper exchange path
                    amount_after_first_trade = (1 / first_leg_price) * second_leg_price
                    amount_after_second_trade = (1 / second_leg_price) * third_leg_price
                    final_amount = amount_after_second_trade

                    expected_return = final_amount
                    net_expected_return = expected_return - (TRADING_FEE * 3)  # Adjust for fees

                    # Print all paths for debugging
                    print(f"🔍 Checking Arbitrage Path: {route[0]} ➝ {route[1]} ➝ {route[2]}")
                    print(f"    ✅ Expected Return: {expected_return:.4%}")
                    print(f"    ✅ Net Profit After Fees: {net_expected_return:.4%}")

                    # Detect and log only profitable trades
                    if net_expected_return > 1.005:  # Ensure profit > 0.5%
                        print(f"✅ VALID Arbitrage Detected on Bybit")
                        print(f"    ✅ Trade Path: {route[0]} ➝ {route[1]} ➝ {route[2]}")
                        print(f"    ✅ Expected Return: {expected_return:.4%}")
                        print(f"    ✅ Net Profit After Fees: {net_expected_return:.4%}")
                        print("-" * 50)
                        log_arbitrage(f"{route[0]} ➝ {route[1]} ➝ {route[2]}", expected_return, net_expected_return)
            except Exception:
                continue

In [ ]:
while True:
    try:
        bybit_tokens = get_tradable_tokens()
        if not bybit_tokens:
            print("❌ No tradable tokens found. Retrying in 10 seconds...")
            time.sleep(10)
            continue

        print("🔄 Checking for arbitrage opportunities...")

        for token in bybit_tokens:
            check_intra_exchange_arbitrage(token)

        check_triangular_arbitrage(bybit_tokens)

        print("✅ Completed this cycle. Waiting for next check...")
        time.sleep(CHECK_INTERVAL)
    except Exception as e:
        print(f"❌ Error in main loop: {e}")
        time.sleep(10)  # Avoid infinite crash loops

✅ Found 427 tradable tokens
🔄 Checking for arbitrage opportunities...
⚠️ Futures market not found for LUNC/USDT (Skipping)
⚠️ Futures market not found for SHIB/USDT (Skipping)
⚠️ Futures market not found for FTT/USDT (Skipping)
⚠️ Futures market not found for STETH/USDT (Skipping)
⚠️ Futures market not found for GENE/USDT (Skipping)
⚠️ Futures market not found for PTU/USDT (Skipping)
⚠️ Futures market not found for TRVL/USDT (Skipping)
⚠️ Futures market not found for NEXO/USDT (Skipping)
⚠️ Futures market not found for REN/USDT (Skipping)
⚠️ Futures market not found for TEL/USDT (Skipping)
⚠️ Futures market not found for IZI/USDT (Skipping)
⚠️ Futures market not found for SIDUS/USDT (Skipping)
⚠️ Futures market not found for VPAD/USDT (Skipping)
⚠️ Futures market not found for DAI/USDT (Skipping)
⚠️ Futures market not found for MIX/USDT (Skipping)
⚠️ Futures market not found for PSTAKE/USDT (Skipping)
⚠️ Futures market not found for DOME/USDT (Skipping)
⚠️ Futures market not found for 